In [1]:
#https://projects.ncsu.edu/crsc/reports/ftp/pdf/crsc-tr06-19.pdf
#https://github.com/lbenz730/world_cup
import pandas as pd
import numpy as np

In [2]:
int_games = pd.read_csv("https://raw.githubusercontent.com/lbenz730/world_cup/master/international_soccer_game_data.csv")

In [3]:
int_games['date'] = pd.to_datetime(int_games['date'])

In [4]:
int_games['recent'] = int_games['date'] - pd.datetime.now().date()
int_games['GD'] = int_games['home_score'] - int_games['away_score']

In [5]:
pr_alo = True

In [6]:
if not pr_alo:
    home_games = int_games[['home_team','away_team','tournament','neutral','recent','date','GD']]
    away_games = int_games[['away_team','home_team','tournament','neutral','recent','date','GD']]
    home_games['loc'] = 'H'
    away_games['loc'] = 'A'
    away_games['GD'] = -1 * away_games['GD']
    col_rename = ['team','opponent','tourney','neutral','recent','date','GD','location']
    away_games.columns = col_rename
    home_games.columns = col_rename
    all_games = pd.concat([away_games,home_games])
    all_games.loc[all_games['neutral'],'location'] = 'N'
    all_games.reset_index(inplace=True)
    relevant_games = all_games[all_games['date'].dt.year > 2014]

In [7]:
if pr_alo:
    select_df = int_games[['home_team','away_team','tournament','neutral','recent','date','GD']]
    col_renamev2 = ['team','opponent','tourney','neutral','recent','date','GD']
    select_df.columns = col_renamev2
    relevant_games = select_df[select_df['date'].dt.year>2014]

In [8]:
def tourney_weights(t_name):
    # WC - 4
    # Continental Cup & Confederations Cup - 3
    three_list = ["AFC Asian Cup","AFC Challenge Cup","African Cup of Nation",
                  "CFU Caribbean Cup","CONCACAF Championship","Gold Cup","Oceania Nations Cup",
                  "UAFA Cup","UEFA Euro","Confederations Cup"]
    # WC Qualifying & CC Qualifying - 2.5
    two_list = ["FIFA World Cup qualification","AFC Asian Cup qualification","AFC Challenge Cup qualification",
                "African Cup of Nations qualification","CFU Caribbean Cup qualification",
                "CONCACAF Championship qualification","Gold Cup qualification",
                "Oceania Nations Cup qualification","UAFA Cup qualification","UEFA Euro qualification"]
    # all others 1
    if t_name == 'FIFA World Cup':
        return 4+1
    elif t_name in three_list: 
        return 3+1
    elif t_name in two_list:
        return 2.5+1
    else:
        return 1+1

In [9]:
relevant_games['weights'] = relevant_games['tourney'].apply(tourney_weights)

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
gd_group = relevant_games.groupby(['weights','team','opponent'])['GD'].agg('sum')

In [11]:
group_scores = gd_group.reset_index()

In [12]:
test = group_scores[group_scores['GD'] == 0]
test['GD'] = -(0.5)
add_values = group_scores.append(test)
add_values.reset_index(inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
add_values.loc[add_values['GD'] == 0,'GD'] = 0.5

In [14]:
def create_from(row):
    if row['GD']>0:
        return row['team']
    else:
        return row['opponent']
def create_to(row):
    if row['GD']<0:
        return row['team']
    else: 
        return row['opponent']

In [15]:
from_scores = add_values
from_scores['from'] = from_scores.apply(create_from, axis=1)
from_scores['to'] = from_scores.apply(create_to,axis=1)

In [16]:
from_scores['GD'] = np.absolute(from_scores['GD'])
final_graph_df = from_scores[['from','to','GD','weights']]

In [17]:
final_graph_df.reset_index(inplace=True)

In [18]:
final_graph_df['weighted_GD'] = np.log(final_graph_df['weights']) * final_graph_df['GD']

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
simple_group_df = final_graph_df.groupby(['from','to'])['weighted_GD'].agg('sum')

In [20]:
simple_df = simple_group_df.reset_index()

### Final Start Page Rank

In [21]:
import networkx as nx

In [22]:
G=nx.from_pandas_dataframe(simple_df, 'from', 'to', 'weighted_GD')

In [23]:
w_ng = nx.pagerank(G)

In [24]:
sorted_w = sorted(w_ng.items(), key=lambda x: x[1],reverse=True)